### EEG-based classification of imagined digits using a recurrent neural network  


In [1]:
import numpy as np
import numpy.fft as fft
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import signal
import sklearn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision
import torchaudio
from torchvision import transforms
import cv2
import time, sys, os
from keras.models import load_model
import importlib

from utils import MindBigData, GetDataSet, GetDataLoaders, GetDataLoadersEEGImages, GetDataAndPreProcess
import MultilayerBidirectionalRNN
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical


seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%matplotlib inline


C:\Users\ryoii\AppData\Roaming\Python\Python310\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


### Signal Acquisition & Preprocessing

In [2]:
def getData(dataName):
    """
    Load EEG data from specified data path, with specified number of samples per digit.
    - GetDataSet : standardization, min-max scaling within std 
        x : 振幅 in microVolts
        y : target digit
    - Preprocess : butterworth filter(highpass 0.1Hz + 平滑化),
    notch filter(ノイズ除去 50Hz), trim first 32 samples
    """

    dataPath = os.path.join(os.path.dirname(os.getcwd()), dataName)

    x_raw, x_preprocessed, x_transformed, x_standardized, y = GetDataAndPreProcess(input_file=dataPath, samples_per_digit=10000)

    return x_raw, x_preprocessed, x_transformed, x_standardized, y

In [3]:
"""
Prepare data 80%:10%:10%
"""
# Split the data and convert y to one-hot encoding
def split_data(x, y, num_classes=10):
    y_one_hot = to_categorical(y, num_classes=num_classes)

    # Split the data into training (80%) and temp (20% for validation and test)
    x_train, x_temp, y_train, y_temp = train_test_split(x, y_one_hot, test_size=0.2, random_state=42)

    # Split the temp data (20%) into validation (10%) and test (10%)
    x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)
    
    return x_train, x_val, x_test, y_train, y_val, y_test

### Deep Learning Classifier

In [8]:
def train(x_train, y_train, x_val, y_val):
    importlib.reload(MultilayerBidirectionalRNN)
    # Initialize the model
    model = MultilayerBidirectionalRNN.MultilayerBidirectionalLSTM(input_shape=(14, 256), num_classes=10, learning_rate=0.001)
    model.summary()
    # Train the model and save the best one
    model.train(x_train=x_train, y_train=y_train, x_val=x_val, y_val=y_val, batch_size=1024, epochs=100, checkpoint_path='best_model_dropout.keras')

    return model

def evaluate(model, x_test, y_test):
    return model.evaluate(x_test, y_test)

In [5]:
dataName = "EP/EP1.01.txt"
# dataName = "MUSE/MU.txt"
x_raw, x_preprocessed, x_transformed, x_standardized, y = getData(dataName=dataName)

[6482, 6324, 6450, 6582, 6309, 6527, 6486, 6296, 6497, 6517]
PreProcess - complete
Wavelet transformation - complete
Standardization - complete


In [6]:
x_train, x_val, x_test, y_train, y_val, y_test = split_data(x_standardized, y)
print(f'shape of x_train : {x_train.shape}')
print(f'shape of y_train : {y_train.shape}')
print(f'shape of x_val : {x_val.shape}')
print(f'shape of y_val : {y_val.shape}')
print(f'shape of x_test : {x_test.shape}')
print(f'shape of y_test : {y_test.shape}')

shape of x_train : (51576, 14, 256)
shape of y_train : (51576, 10)
shape of x_val : (6447, 14, 256)
shape of y_val : (6447, 10)
shape of x_test : (6447, 14, 256)
shape of y_test : (6447, 10)


In [15]:
model = train(x_train, y_train, x_val, y_val)

this is 1 dropout version


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_12                │ (None, 14, 512)        │     1,050,624 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_13                │ (None, 14, 256)        │       656,384 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_14                │ (None, 128)            │       164,352 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,872,650 (7.14 MB)

 Trainable params: 1,872,650 (7.14 MB)

 Non-trainable params: 0 (0.00 B)

val_accuracy version
Epoch 1/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1003 - loss: 2.3293 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00
Epoch 1: val_accuracy improved from inf to 0.10904, saving model to best_model_dropout.keras
51/51 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.1004 - loss: 2.3290 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00 - val_accuracy: 0.1090 - val_loss: 2.3031 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 2/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1013 - loss: 2.3044 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00
Epoch 2: val_accuracy did not improve from 0.10904
51/51 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.1013 - loss: 2.3044 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00 - val_accuracy: 0.1134 - val_loss: 2.2994 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 3/100
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.1117 - loss: 2.2992 - precision_2: 0.0000e+00 - recall_2: 0.000

In [12]:
model = load_model('best_model.keras')

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)

# Print the test loss and accuracy
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

202/202 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.1704 - loss: 2.2515
Test Loss: 2.2501940727233887
Test Accuracy: 0.1636420041322708
